# Tâche 3 : Question-réponse avec GPT‑2 avec poursuite du pré-entraînement sur un corpus de Sherlock Holmes

**Objectifs**

Évaluer la qualité des réponses d’un modèle de langage **pré‑entraîné** (version **GPT-2 Medium (355M)** sur Hugging Face) lorsqu’on lui pose des questions sur un sujet vu au pré-entraînement.
Dans ce *notebook*, vous poursuivez le pré-entraînement du modèle avec des textes de l'univers de *Sherlock Holmes*.  Comme pour la tâche 2, on doit construire un *prompt* minimal, générer des réponses avec le nouveau modèle, et évaluer la pertinence des résultats. Plusieurs de ces fonctions sont rendues disponibles.

**Objectifs d’apprentissage**
1. Poursuivre le préentraînement d'un modèle pré‑entraîné (Hugging Face) sur un corpus de taille moyenne.
2. Comprendre et expliquer les **limites et apports du pré‑entraînement** sur des textes pertinents au domaine des questions.

Tout comme pour la tâche 2, les **questions** pour évaluer le modèle vous sont fournies. Vous devez comprendre le format des questions chargées en mémoire. La **liste de livres** à utiliser pour le pré-entraînement et la **fonction** pour les monter en mémoire sont également disponibles.

NOTE: Il est important de sauvegarder le modèle pré-entraîné dans cette tâche (ainsi que son tokenizer) car nous les réutilisons pour la tâche 4.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [ ]:
#!pip install datasets
#!pip install accelerate
#!pip install 'transformers[torch]'
#!pip3 install torch torchvision

In [ ]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécessite pas un plus grand contexte
model_name = "gpt2-medium"

In [ ]:
from datasets import Dataset
from transformers import pipeline, Trainer
import os
import json

import re
import requests

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # GPT-2 n'en a pas par défaut
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.eos_token_id

    return tokenizer, model

In [ ]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")


## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) :
* Construire un prompt à l’aide de la fonction **build_prompt** (rendu disponible).
* Utiliser le modèle (via un pipeline de génération de texte passé en argument) pour générer une réponse à une question.
* Retourner la réponse générée par le modèle.  

Points importants à souligner:
* La fonction *process_entry* doit retourner uniquement la réponse générée par le modèle (sans le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

In [ ]:
def build_prompt(entry):
    return entry.get("question", "")

### Description des paramètres de génération:
- **max_new_tokens=100** : Limite la longueur de la réponse à 100 tokens pour éviter des réponses trop longues
- **do_sample=True** : Active l'échantillonnage probabiliste pour des réponses plus variées et naturelles
- **temperature=0.7** : Contrôle la créativité (0.7 offre un bon équilibre entre cohérence et diversité)
- **top_p=0.9** : Considère les tokens dont la probabilité cumulative atteint 90%
- **top_k=100**: Limite la sélection aux 100 tokens les plus probables à chaque étape
- **pad_token_id** : Nécessaire pour éviter les warnings avec GPT-2

In [ ]:
def process_entry(entry, prompt_builder, generator):
    # Construit le prompt à partir de l'entrée
    prompt = prompt_builder(entry)

    # Génére la réponse avec le modèle
    result = generator(
        prompt,
        max_new_tokens=100,       # Longueur maximale de la réponse
        do_sample=True,           # Active l'échantillonnage probabiliste
        temperature=0.7,          # Contrôle la créativité
        top_p=0.9,                # Échantillonnage nucleus
        top_k=100,                # Échantillonnage top-k
        pad_token_id=generator.tokenizer.eos_token_id  # Évite les warnings
    )

    # Extrait le texte généré complet
    generated_text = result[0]['generated_text']

    # Retourne uniquement la réponse (enleve le prompt)
    answer = generated_text[len(prompt):].strip()
    return answer

In [ ]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry(entry, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Poursuite du pré-entraînement du modèle GPT-2 (à compléter)

Complétez le code suivant afin de poursuivre le préentraînement de GPT2 sur des textes de *Sherlock Holmes*.

Les étapes à suivre sont de :
* Télécharger le contenu des livres (on rend la fonction disponible)
* Créer un *dataset* (version Hugging Face) d'entraînement à partir de ce contenu
* Tokeniser ce *dataset*
* Faire le pré-entraînement du modèle sur le *dataset* avec la classe ***Trainer*** de Hugging Face
* Faire la sauvegarde du nouveau modèle dans un répertoire (voir *model_path*)

In [ ]:
books = {
    "The Sign of the Four": "https://www.gutenberg.org/files/2097/2097-0.txt",
    "The Adventures of Sherlock Holmes": "https://www.gutenberg.org/files/1661/1661-0.txt",
    "The Memoirs of Sherlock Holmes": "https://www.gutenberg.org/files/834/834-0.txt",
    "The Hound of the Baskervilles": "https://www.gutenberg.org/files/2852/2852-0.txt",
    "His Last Bow": "https://www.gutenberg.org/files/2350/2350-0.txt",
    "The Case-Book of Sherlock Holmes": "https://www.gutenberg.org/files/221/221-0.txt"
}

def download_sherlock_dataset(books_to_process):
    data = []

    for title, url in books_to_process.items():
        response = requests.get(url)

        if response.status_code == 200:
            text = response.text

            # Cette expression est plus robuste que ce qui était attendu dans le premier travail pratique
            header_regex = r"(?s)^.*?\*{3}\s*START OF\b.*?\r?\n?\*{3}\s*\r?\n"
            header_pattern = re.compile(header_regex, flags=0)
            clean_text = header_pattern.sub("", text)

            toc_regex = r"(?ims)^\s*contents\s*$.*?^\s*$"
            toc_pattern = re.compile(toc_regex, flags=0)
            clean_text = toc_pattern.sub("", clean_text)

            regex_license_llm = r"(?im)^\s*\*{3} END OF\b.*[\s\S]*\Z"
            license_llm_pattern = re.compile(regex_license_llm, flags=0)
            clean_text = license_llm_pattern.sub("", clean_text)

            # To make it simpler to learn text without learning new lines
            clean_text = re.sub(r"\r\n\r\n", "\n", clean_text)
            clean_text = re.sub(r"\r\n", " ", clean_text)

            data.append(clean_text)
            print(f"Downloaded: {title}")
        else:
            print(f"Failed to download: {title}")

    return "\n".join(data)

Création du *dataset* d'entraînement.

In [ ]:
sherlock_text = download_sherlock_dataset(books_to_process=books)
text_lines = [line.strip() for line in sherlock_text.split('\n') if line.strip()]


from datasets import Dataset
sherlock_dataset = Dataset.from_dict({"text": text_lines})
print(f"Nombre de lignes dans le dataset : {len(sherlock_dataset)}")

Downloaded: The Sign of the Four
Downloaded: The Adventures of Sherlock Holmes
Downloaded: The Memoirs of Sherlock Holmes
Downloaded: The Hound of the Baskervilles
Downloaded: His Last Bow
Downloaded: The Case-Book of Sherlock Holmes
Nombre de lignes dans le dataset : 11052


Tokénisation du *dataset* d'entraînement.

In [ ]:
tokenizer, model = load_model(model_name)

# 1) Tokenisation
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        add_special_tokens=False,
        return_attention_mask=False,  # évite la colonne attention_mask
    )

tokenized = sherlock_dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

# 2) Regroupement en blocs
block_size = max_length
def group_texts(examples):
    """Concatène tous les tokens et les regroupe en blocs de taille block_size"""
    ids = sum(examples["input_ids"], [])
    total = (len(ids) // block_size) * block_size # Nombre total de tokens utilisables
    input_ids = [ids[i:i+block_size] for i in range(0, total, block_size)]
    return {"input_ids": input_ids, "labels": input_ids}

tokenized_dataset = tokenized.map(group_texts, batched=True)
print(f"Nombre d'exemples après regroupement : {len(tokenized_dataset)}")
print(f"Taille de chaque bloc : {block_size} tokens")


Map:   0%|          | 0/11052 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1175 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/11052 [00:00<?, ? examples/s]

Nombre d'exemples après regroupement : 2737
Taille de chaque bloc : 256 tokens


Ajouter dans les cellules suivantes le code dont vous avez besoin pour poursuivre le pré-entraînement du modèle.

In [ ]:
model_path = "gpt2-sherlock-lm"  # Répertoire où sauvegarder le nouveau modèle et le tokenizer

from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# collator causal LM (pas de MLM pour GPT-2)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    per_device_train_batch_size=batch_size,  # batch_size=5
    num_train_epochs=3,       # Nombre d'époques d'entraînement
    learning_rate=5e-5,       # Taux d'apprentissage
    save_steps=500,           # Sauvegarde tous les 500 steps
    logging_steps=100,        # Logge tous les 100 steps
    fp16=True,                # Accélère l'entraînement
    report_to="none"          # Désactive wandb
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
100,2.909000
200,2.787200
300,2.713100
400,2.664900
500,2.657800
600,2.540700
700,2.424800
800,2.414800
900,2.406200
1000,2.398600


TrainOutput(global_step=1644, training_loss=2.4718082084562947, metrics={'train_runtime': 1016.0095, 'train_samples_per_second': 8.082, 'train_steps_per_second': 1.618, 'total_flos': 3812780701384704.0, 'train_loss': 2.4718082084562947, 'epoch': 3.0})

Pour conclure cette section, sauvegardez le nouveau modèle et le *tokenizer* afin de les réutiliser dans la tâche 4.







In [ ]:
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('gpt2-sherlock-lm/tokenizer_config.json',
 'gpt2-sherlock-lm/special_tokens_map.json',
 'gpt2-sherlock-lm/vocab.json',
 'gpt2-sherlock-lm/merges.txt',
 'gpt2-sherlock-lm/added_tokens.json',
 'gpt2-sherlock-lm/tokenizer.json')

## 4. Génération de réponses avec le nouveau modèle GPT-2

Exécutez la cellule suivante pour générer les réponses aux questions avec le modèle que vous venez de pré-entraîner sur des textes de *Sherlock Holmes*.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail.

In [ ]:
questions = "data/questions_sherlock.json"
out_file_name = "pretrained_gpt2_answers.txt"

results = test_on_questions(prompt_builder=build_prompt, model_path=model_path, question_file=questions, out_file_name=out_file_name)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Q0: Where do Sherlock Holmes and Dr. Watson live?
A: ”“In a very old, very bad, very shabby little house called the Baker Street Station.”“And what do they do there?”“They get their information from the station-master and from the post-master.”“And how do they get their information?”“From the post-master.”“How did you get it?”“Well, I was walking round the corner on my
Expected:221B Baker Street, London.
------------------------------------------------------------
Q1: Who is Sherlock Holmes' loyal friend and chronicler?
A: ”“Well, he’s my friend, but I don’t know him very well. He is a little retiring, and I don’t know that he would want me to go out of my way to meet him.”“But you have never met him?”“I have not.”“Well, then, how do you know that he is a friend?”“Well, I have heard of him
Expected:Dr. John H. Watson.
------------------------------------------------------------
Q2: Who is considered 'The Woman' by Sherlock Holmes?
A: ”“My dear fellow, there are many, many       who kno

#### Remarque
Le pré-entraînement sur Sherlock Holmes améliore le vocabulaire et le style des réponses (mention de Baker Street, violin, London), mais les réponses restent trop longues et imprécises. Le modèle génère des dialogues fictifs plutôt que des réponses directes, ce qui montre la nécessité d'un affinage par instructions.

## 5. Analyse des résultats

### 5.1 Évaluation quantitative (à compléter)

In [ ]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [ ]:
def evaluation_generation(results):
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_questions = len(results)

    # Calcule les métriques pour chaque question
    for i, question, answer, expected_answer in results:
        precision, recall, f1 = evaluate_f1(expected_answer, answer)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

    # Calcule les moyennes
    eval = {
        'precision_moyenne': total_precision / num_questions,
        'rappel_moyen': total_recall / num_questions,
        'f1_moyen': total_f1 / num_questions,
        'nombre_questions': num_questions
    }
    return eval

In [ ]:
eval = evaluation_generation(results)
print(eval)

{'precision_moyenne': 0.003935810574107621, 'rappel_moyen': 0.0861904761904762, 'f1_moyen': 0.007400289606916743, 'nombre_questions': 50}


**Question :** Que pensez-vous de cette évaluation ?

Les résultats sont très faibles (F1 = 0.74%, précision = 0.39%, rappel = 8.6%), mais cela ne signifie pas que le modèle n'a rien appris. Ces scores bas s'expliquent par l'inadéquation de la métrique F1 avec ce type de tâche : le modèle génère des réponses très longues et narratives alors que les réponses attendues sont courtes.

Par exemple, pour "What instrument does Holmes play?", le modèle répond "I play a violin. But he can play a harp..." au lieu de simplement "The violin". L'information est présente mais noyée dans du texte superflu.

Cette évaluation montre surtout que le pré-entraînement seul ne suffit pas : le modèle a appris le style et le vocabulaire de Sherlock Holmes, mais il a besoin d'un affinage par instructions pour apprendre à répondre de manière concise et directe.

## 5. Analyse qualitative (à faire)

Rédigez **5 à 15 phrases** présentant vos observations et expliquant pourquoi, selon vous, le modèle fournit ce type de réponses.

Vous pouvez ajouter des cellules au besoin.

> Cette étape prépare le terrain pour la tâche 4.


Le modèle GPT-2 pré-entraîné sur les textes de Sherlock Holmes montre des améliorations notables par rapport au modèle de base (tâche 2), mais présente encore des limites importantes. On observe que le modèle a assimilé le vocabulaire spécifique de l'univers Sherlock Holmes : il mentionne maintenant correctement des éléments comme "Baker Street", "violin", "revolver" et "London", ce qui était absent dans les réponses du modèle de base.

Cependant, le modèle génère systématiquement des réponses beaucoup trop longues et verbeuses au lieu de fournir des réponses factuelles courtes et directes. Cette verbosité s'explique par le fait que le modèle a été entraîné sur des romans narratifs où les informations sont présentées sous forme de dialogues et de descriptions élaborées, et non sous forme de questions-réponses concises. Le modèle a donc appris à "raconter des histoires" plutôt qu'à répondre directement aux questions.

On remarque également que le modèle éprouve des difficultés à fournir des noms propres précis (comme "Mycroft Holmes", "Irene Adler" ou "Professor Moriarty") et préfère générer des descriptions génériques ou inventer des noms alternatifs. Cela suggère que le simple pré-entraînement sur les textes ne suffit pas pour extraire et mémoriser des faits précis de manière structurée.

En conclusion, le pré-entraînement a permis d'acquérir le style, le vocabulaire et certaines connaissances générales sur l'univers de Sherlock Holmes, mais le modèle manque encore de la capacité à produire des réponses courtes, précises et factuelles. Ces observations indiquent clairement la nécessité d'une étape d'affinage par instructions (tâche 4) qui apprendra au modèle à répondre de manière structurée en format question-réponse, plutôt que de générer des récits narratifs.